Zalecamy nie czytać notatników na githubie, ze względu na źle wyświetlające się wizualizacje i brak możliwości uruchamiania kodu. Polecamy otworzyć notatnik w google colab.

# **Olimpiada AI - kurs wprowadzający 2025 - Wykład 05B**

"Spadek wzdłuż gradientu (gradient descent)"

##### Kod do wizualizacji
(to jest kod pomocniczy, nie trzeba analizować, używany tylko do generacji obrazka)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

class FunctionTracker:
    def __init__(self, func):
        self.func = func
        self.x_values = []
        self.y_values = []
        self.f_values = []

    def add_point(self, x, y):
        # Konwersja tensorów PyTorch na liczby, jeśli to konieczne
        if hasattr(x, 'item'): x = x.item()
        if hasattr(y, 'item'): y = y.item()

        self.x_values.append(x)
        self.y_values.append(y)
        self.f_values.append(self.func(x, y))

    def plot(self):
        # --- WIDOK 3D (PLOTLY) ---
        x_plot = np.linspace(-2.5, 2.5, 100)
        y_plot = np.linspace(-2.5, 2.5, 100)
        X, Y = np.meshgrid(x_plot, y_plot)

        # Obsługa funkcji, która może przyjmować tensory lub numpy
        try:
            Z = self.func(X, Y)
        except:
            # Fallback dla funkcji PyTorchowej, jeśli dostanie siatkę numpy
            Z = np.zeros_like(X)
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    val = self.func(np.array([X[i,j], Y[i,j]]))
                    if hasattr(val, 'item'): val = val.item()
                    Z[i,j] = val

        fig = go.Figure(data=[
            go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.8),
            go.Scatter3d(
                x=self.x_values,
                y=self.y_values,
                z=self.f_values,
                mode='lines+markers',
                marker=dict(size=4, color='red'),
                line=dict(color='red', width=2)
            )
        ])

        fig.update_layout(
            title='Wizualizacja 3D: Wspinaczka na szczyt',
            scene=dict(
                xaxis_title='x',
                yaxis_title='y',
                zaxis_title='Wysokość',
                aspectmode='cube'
            ),
            autosize=False,
            width=800,
            height=600,
        )
        fig.show()

        # --- WIDOK 2D (MATPLOTLIB - MAPA POZIOMICOWA) ---
        plt.figure(figsize=(8, 6))
        plt.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.8)
        plt.colorbar(label='Wysokość terenu')
        plt.plot(self.x_values, self.y_values, 'r.-', label='Trasa turysty')
        plt.scatter(self.x_values[0], self.y_values[0], color='white', marker='x', s=100, label='Start', zorder=5)
        plt.scatter(self.x_values[-1], self.y_values[-1], color='black', marker='x', s=100, label='Koniec', zorder=5)
        plt.title("Widok z góry (Mapa poziomicowa)")
        plt.xlabel("Współrzędna X")
        plt.ylabel("Współrzędna Y")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

def illustration():
    fig, ax = plt.subplots(figsize=(10, 8))

    # Współrzędne kluczowych punktów
    hill1 = (0, 0)
    hill2 = (1, 1)
    turysta = (1.6, 0.4)

    # Generowanie siatki terenu
    x = np.linspace(-2, 3, 400)
    y = np.linspace(-2, 3, 400)
    X, Y = np.meshgrid(x, y)

    # Funkcja terenu (identyczna jak w dalszej części)
    # Szczyt 1 w (0,0), Szczyt 2 w (1,1)
    Z = np.exp(-X**2 - Y**2) + np.exp(-(X - 1)**2 - (Y - 1)**2) / 2

    # Rysowanie mapy ciepła
    cf = ax.contourf(X, Y, Z, levels=50, cmap='plasma')
    plt.colorbar(cf, ax=ax, label='Wysokość terenu')

    # Zaznaczenie punktów
    ax.scatter(*hill1, color='red', s=150, label='Główny Szczyt (0,0)', edgecolors='white', zorder=5)
    ax.scatter(*hill2, color='blue', s=100, label='Mniejsza Górka (1,1)', edgecolors='white', zorder=5)
    ax.scatter(*turysta, color='white', s=120, label='Ty (Start)', marker='X', edgecolors='black', zorder=5)

    # Podpisy
    ax.text(hill1[0], hill1[1]-0.3, "Główny Szczyt", color='white', ha='center', fontweight='bold')
    ax.text(hill2[0], hill2[1]+0.2, "Mniejsza Górka", color='white', ha='center', fontweight='bold')
    ax.text(turysta[0], turysta[1]-0.25, "Ty", color='white', ha='center', fontweight='bold')

    # Okrąg symbolizujący zasięg czujnika (epsilon)
    epsilon_radius = 0.3
    circle = plt.Circle(turysta, epsilon_radius, color='white', fill=False, linestyle='--', linewidth=2, label='Widoczność')
    ax.add_artist(circle)

    # Strzałki symbolizujące badanie terenu
    arrow_props = dict(head_width=0.08, color='white', alpha=0.8)
    ax.arrow(turysta[0], turysta[1], epsilon_radius, 0, **arrow_props)
    ax.arrow(turysta[0], turysta[1], -epsilon_radius, 0, **arrow_props)
    ax.arrow(turysta[0], turysta[1], 0, epsilon_radius, **arrow_props)
    ax.arrow(turysta[0], turysta[1], 0, -epsilon_radius, **arrow_props)

    ax.set_aspect('equal')
    ax.set_xlabel('Współrzędna X')
    ax.set_ylabel('Współrzędna Y')
    ax.set_title('Wizualizacja sytuacji początkowej')
    ax.legend(loc='upper right')

    plt.show()

## Turysta i mgła w górach

### Wstęp
Wyobraź sobie, że jesteś turystą w górach. Nie masz mapy, ale i tak chcesz wejść na najwyższy szczyt w okolicy. Jest jednak problem: panuje gęsta mgła i widoczność jest bardzo słaba. Widzisz tylko to, co masz pod nogami.

Nie masz mapy więc nie widzisz od razu w którą stronę iść, ale możesz sobie poradzić inaczej. Masz inne narzędzie - swoje stopy - które działają trochę jak "czujnik".

Stoisz w miejscu i delikatnie badasz teren wokół siebie:
* Stawiasz stopę w prawo - teren opada.
* Stawiasz stopę w lewo - teren się wznosi.
* Stawiasz stopę w przód - teren się wznosi, ale mniej stromo.

Co robisz? Robisz krok w tę stronę, gdzie teren wznosi się najbardziej.
Żeby dojść na szczyt musisz powtarzać w kółko ten sam "algorytm": badasz grunt -> robisz krok -> badasz grunt -> robisz krok.

To właśnie jest Gradient Ascent (lub Descent, gdy schodzimy w dół). To nic innego jak podróżowanie po górach i dolinach we mgle.

In [ ]:
illustration()

#### Matematyczna góra
Zdefiniujmy sobie prawdziwą funkcję (ground truth) która przekształca przestrzeń - dla każdego `x` i`y` - przypisuje wysokość.
W naszym kodzie teren, po którym będziemy chodzić, jest opisany wzorem:
$$ f(x, y) = e^{-x^2-y^2} + \frac{1}{2}e^{-(x-1)^2-(y-1)^2} $$

Niech Cię nie przeraża ten zapis, to po prostu matematyczny opis dwóch "górek"
* Pierwsza (większa) jest w punkcie (0,0).
* Druga (mniejsza) jest w punkcie (1,1).

#### Parametry naszej wspinaczki
W kodzie spotkasz zmienne, które mają swoje odpowiedniki:
1.  `epsilon` (widoczność) – jak daleko możesz sięgnąć stopą, żeby zbadać teren.
2.  `num_iterations` (wytrzymałość) – ile kroków masz siłę zrobić.
3.  `x0` (punkt startowy) – miejsce, w którym zaczynasz swoją wędrówkę

In [ ]:
# Definicja "górskiego terenu" (funkcji)
def f(x, y):
    # To jest wzór na nasz teren z dwoma szczytami
    return np.exp(-x**2 - y**2) + np.exp(-(x - 1)**2 - (y - 2)**2) / 2

# Funkcja symulująca badanie terenu stopą
def find_best_df(f, x, epsilon):
    # Sprawdzamy 4 kierunki: Północ, Południe, Wschód, Zachód
    directions = np.array([[ 0,  1],
                           [ 0, -1],
                           [ 1,  0],
                           [-1,  0]])

    directions = directions * epsilon
    directions = directions + x # Przesuwamy "stopę" w danym kierunku

    # Sprawdzamy wysokość w każdym z tych punktów
    dir_values = np.array([f(dir[0], dir[1]) for dir in directions])

    # Wybieramy kierunek, gdzie jest najwyżej
    best_dir_idx = np.argmax(dir_values)
    best_dir = directions[best_dir_idx]

    # Zwracamy wektor ruchu (gdzie zrobić krok)
    return best_dir - x

### Warianty poruszania się

- Wariant 1 (ŚLIMAK): Ustaw epsilon na bardzo małą liczbę (np. 0.01). Czy turysta zdąży wejść na szczyt w 50 krokach?
- Wariant 2 (KANGUR): Ustaw epsilon na dużą liczbę (np. 0.5). Czy turysta trafi w szczyt, czy będzie go przeskakiwać?
- Wariant 3 (PUŁAPKA): Ustaw punkt startowy x0 na [1.6, 2.5]. Na który szczyt wejdzie turysta? Czy to ten najwyższy?

In [ ]:
x0 = np.array([1.6, 2.5])    # Punkt startowy
num_iterations = 30          # Liczba kroków
epsilon = 0.1               # Długość kroku

function_tracker = FunctionTracker(f)

x = x0
# Pętla wspinaczki
for i in range(num_iterations):
    df = find_best_df(f, x, epsilon) # Zbadaj teren
    x = x + df              # Zrób krok

    function_tracker.add_point(x[0], x[1])

function_tracker.plot()

#### Wnioski

1.  Długość kroku ma znaczenie: Jak idziesz za wolno (mały epsilon), to możesz nigdy na niego nie dojść. Jak idziesz zbyt szybko (duży epsilon), to możesz przeskoczyć szczyt i nigdy na nim nie stanąć.
2.  Pułapki lokalne: Jeśli zaczniesz wspinaczkę bliżej mniejszej górki, Twój algorytm zaprowadzi Cię na nią i już nigdy z niej nie zejdziesz. Będziesz na szczycie i będziesz myślał, że osiągnąłeś swój cel. Ale to nie jest najwyższy szczyt (Globalne Maximum), tylko lokalny pagórek (Lokalne Maximum). To wielki problem - i tak napawdę nierozwiązywalny - przy treningu modeli sztucznej inteligencji! (Jednak jest trochę trików, które pozwalają nam wydostać się czasami z takich lokalnych pagórków - o nich będzie w następnych wykładach.)

Kilka słów wyjaśnienia:
- w machine learningu przeważnie *minimalizujemy* funkcję, więc zwykle mówi się i spotyka się nazwę *gradient descent*;
- powyższy algorytm jest jedynie uproszczoną wersją gradient ~~descentu~~ ascentu, w rzeczyiwstości wykorzystuje się mechanizm `autograd` pokazany poniżej


## Automatyczny Gradient Descent (autograd)

W poprzedniej części musieliśmy zbadać teren, sprawdzając wszystkie 4 kierunki w pętli. Jest jednak dużo szybszy sposób! Możemy również uzyskać dużo dokładniejszy kierunek niż "północ", "południe", "wschód", "zachód". Ten prawdziwy kierunek, to właśnie prawdziwy *gradient*, czyli kierunek największego wzrostu funkcji / kierunek w którym jest najbardziej strormo.
PyTorch jest w stanie właśnie takie prawdziwe kierunki, czyli gradienty, wyznaczyć precyzyjnie, dzięki mechanizmowi automatycznego różniczkowania, zwanego `autograd`em, z którego korzysta.

### **Gradienty**
W przypadku Pytorcha każdy kolejny krok jest wyliczany na podstawie *gradientu* $\nabla f(x)$ funkcji $f$ w punkcie $x$. Formalna definicja gradientu to


$$
\nabla f(x, y) = \left( \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} \right),
$$
gdzie $\frac{\partial f}{\partial x}$ to pochodna funkcji $f(x,y)$ po zmiennej $x$, a $\frac{\partial f}{\partial y}$ to *pochodna* funkcji $f(x,y)$ po zmiennej $y$.

Jeśli ktoś jeszcze nie uczył się obliczania pochodnych, to zupełnie nie szkodzi. Nie będziemy musieli zagłębiać się w nie bardziej niż to, co pokazane w tym wykładzie.



### Learning Rate
W PyTorchu `epsilon` zmienia nazwę na `learning_rate` (tempo uczenia).
* Mały Learning Rate = algorytm wolno się uczy, ale dokładnie.
* Duży Learning Rate - algorytm szybko się uczy, ale robi błędy i "przeskakuje"

In [ ]:
import torch

# Definicja góry w PyTorchu (używamy torch.exp zamiast np.exp)
def f_torch(x):
    return torch.exp(-x[0]**2 - x[1]**2) + torch.exp(-(x[0] - 1)**2 - (x[1] - 2)**2) / 2

# Używamy wersji numpy tylko do rysowania wykresu
def f_np(x, y):
    return np.exp(-x**2 - y**2) + np.exp(-(x - 1)**2 - (y - 2)**2) / 2

function_tracker = FunctionTracker(f_np)

# Parametry
x0 = torch.tensor([1.6, 0.4], dtype=torch.float, requires_grad=True)  # Start
learning_rate = 0.1   # Długość kroku
num_iterations = 100  # Liczba kroków

x = x0
# Pętla Gradient Descent (Wspinaczki)
for i in range(num_iterations):
    # 1. Oblicz wysokość w obecnym punkcie
    value = f_torch(x)

    # 2. Magia PyTorcha: Oblicz kierunek najszybszego spadku/wzrostu (Gradient)
    value.backward()

    # 3. Zrób krok w stronę wskazaną przez Pytorch
    with torch.no_grad():
        x += learning_rate * x.grad  # x.grad to nasza strzałka z GPS-a

        # Wyzeruj GPS przed kolejnym pomiarem (techniczny wymóg PyTorcha)
        x.grad.zero_()

    function_tracker.add_point(x[0], x[1])

function_tracker.plot()

# Symulacja Autograd

Pojawia się więc pytanie: Jak PyTorch wie, gdzie jest stromo? Żeby przekonać się sami napiszemy własną, uproszczoną wersję takiego "kalkulatora gradientu".

Nazwiemy ją `SymulacjaAutograd`.

Funkcja ta działa tak:
1. Bierze obecną pozycję.
2. Przesuwa się o mikroskopijny kawałek (`epsilon`) wzdłuż osi X i patrzy, o ile zmieniła się wysokość.
3. Przesuwa się o mikroskopijny kawałek wzdłuż osi Y i patrzy, o ile zmieniła się wysokość.
4. Składa te dwie informacje w jedną strzałkę (wektor).

Przesuwamy się teraz po ukosie!

In [ ]:
def SymulacjaAutograd(f, x, epsilon=0.01):
    grad = np.zeros(len(x))
    # Dla każdego wymiaru (tutaj X i Y)
    for i in range(0, len(x)):
        # Stwórz przesunięcie tylko w jednym kierunku
        przesuniecie = np.zeros(len(x))
        przesuniecie[i] = 1 # wektor jednostkowy np. [1, 0]

        # WZÓR: (Wysokość po kroku - Wysokość przed krokiem) / długość kroku
        # To mówi nam: "Jak bardzo stromo jest w tym kierunku?"
        nachylenie = (f(x + epsilon * przesuniecie) - f(x)) / epsilon

        grad[i] = nachylenie
    return grad

In [ ]:
# Sprawdźmy naszą zaimplementowaną symulację
x0 = np.array([1.6, 0.4])
learning_rate = 0.1
num_iterations = 100

function_tracker = FunctionTracker(lambda x, y: f_np(x, y))

df_list = [] # Tu będziemy zapisywać, jak duże kroki robiliśmy
fx_list = [] # Tu będziemy zapisywać wysokość

x = x0

for i in range(num_iterations):
    x_prev = x

    # Zamiast PyTorcha używamy naszej funkcji!
    Gx = SymulacjaAutograd(lambda v: f_np(v[0], v[1]), x)

    # Krok w górę zgodnie ze wskazaniem czujnika
    x = x + learning_rate * Gx

    df_list.append((f_np(x[0], x[1]) - f_np(x_prev[0], x_prev[1])))
    fx_list.append(f_np(x[0], x[1]))
    function_tracker.add_point(x[0], x[1])

function_tracker.plot()

### Analiza zmęczenia turysty
Spójrzmy na wykres poniżej.
* Linia pomarańczowa (Wysokość): Szybko rośnie na początku, a potem się wypłaszcza. To znaczy, że turysta szybko wszedł na zbocze, a potem już tylko dreptał po płaskim szczycie.
* Linia niebieska (Zmiana wysokości): Na początku skoki są duże, ale pod koniec są bliskie zeru. To moment, gdy turysta stoi na szczycie i każdy krok w bok nie zmienia już jego wysokości.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(len(df_list)), df_list, label='Zysk wysokości w jednym kroku (Gradient)')
plt.plot(range(len(fx_list)), fx_list, label='Aktualna wysokość n.p.m. (Funkcja celu)')

plt.xlabel('Numer kroku (Iteracja)')
plt.xlim(0,50)
plt.ylabel('Wartość')
plt.title('Historia wspinaczki')
plt.legend()
plt.grid(True)
plt.show()